In [ ]:
import Thymio as th
import time
import costmap_path as cp
import cv2

#FIXME: not right original image
original_image = cv2.imread('map.png',0)
costmap, block_size, start, goal, display_image=cp.init(original_image)
path,costmap = cp.update(costmap, start, goal, display_image)
next_goal = path[0]
current_pos = (0,0,0)
camera_position = (0,0,0)
kalman_state = (0,0)
thymio = th.Thymio()
camera_on = True
while True:
    
    #call camera to get image and update 
    
    #call function to get position of thymio from camera
    
    
    #call kallman filter
    #call kalmann prediction to update the sate and get current position in (x,y,theta)
   
    obstacle,detected = th.getObstaclePosition()
    if detected:
        print("Obstacle detected")
        #TODO: use the obstacle position to update the costmap
        path,costmap = cp.update(costmap, start, goal, display_image)
        
        #FIXME:need to set threshold for proximity to obstacle
    if current_pos == next_goal:
        path.pop(0)
        if len(path) == 0:
            print("Goal reached")
            break
        next_goal = path[0]
        
    
    
    
    
    
    
    #call PID controller
    
 





